經小編測試，本範例程式只能在Tensorflow 2.11.0 版本上面執行，因此範例程式一開始會先在 Colab 上替Tensorflow 降版，如下：

In [ ]:
 !pip install --upgrade tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 56.4 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.1
    Uninstalling tensorboard-data-server-0.7.1:
      Successfully uninstalled tensorboard-data-server-0.7.1
  Att

In [ ]:
import tensorflow as tf

print(tf.__version__)

2.11.0


程式6-1

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text \
    import text_to_word_sequence
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

EPOCHS = 2
BATCH_SIZE = 256
INPUT_FILE_NAME = '/content/frankenstein.txt'
WINDOW_LENGTH = 40
WINDOW_STEP = 3
PREDICT_LENGTH = 3
MAX_WORDS = 7500
EMBEDDING_WIDTH = 100


程式6-2

In [ ]:
# 讀取檔案
file = open(INPUT_FILE_NAME, 'r', encoding='utf-8-sig')
text = file.read()
file.close()

# 將文本全轉為小寫、斷字、轉為串列
text = text_to_word_sequence(text)

# 建立訓練樣本 / 正解
fragments = []
targets = []
for i in range(0, len(text) - WINDOW_LENGTH, WINDOW_STEP):
    fragments.append(text[i: i + WINDOW_LENGTH])
    targets.append(text[i + WINDOW_LENGTH])


程式6-3

In [ ]:
# 將單字轉換成數字編號
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='UNK')
tokenizer.fit_on_texts(text)
fragments_indexed = tokenizer.texts_to_sequences(fragments)
targets_indexed = tokenizer.texts_to_sequences(targets)
print(tokenizer.word_index)

# 備妥輸入資料 X、正解 y
X = np.array(fragments_indexed, dtype=np.int64)
y = np.zeros((len(targets_indexed), MAX_WORDS))
for i, target_index in enumerate(targets_indexed):
    y[i, target_index] = 1


{'UNK': 1, 'the': 2, 'and': 3, 'i': 4, 'of': 5, 'to': 6, 'my': 7, 'a': 8, 'in': 9, 'was': 10, 'that': 11, 'me': 12, 'with': 13, 'but': 14, 'had': 15, 'you': 16, 'he': 17, 'which': 18, 'it': 19, 'as': 20, 'not': 21, 'his': 22, 'for': 23, 'by': 24, 'on': 25, 'this': 26, 'from': 27, 'be': 28, 'have': 29, 'her': 30, 'at': 31, 'is': 32, 'when': 33, 'were': 34, 'or': 35, 'your': 36, 'she': 37, 'one': 38, 'an': 39, 'him': 40, 'all': 41, 'they': 42, 'so': 43, 'if': 44, 'will': 45, 'could': 46, 'are': 47, 'would': 48, 'been': 49, 'their': 50, 'we': 51, 'no': 52, 'who': 53, 'more': 54, 'these': 55, 'some': 56, 'should': 57, 'now': 58, 'yet': 59, 'before': 60, 'myself': 61, 'any': 62, 'what': 63, 'upon': 64, 'our': 65, 'do': 66, 'man': 67, 'them': 68, 'its': 69, 'only': 70, 'am': 71, 'into': 72, 'did': 73, 'life': 74, 'every': 75, 'than': 76, 'may': 77, 'then': 78, 'own': 79, 'first': 80, 'father': 81, 'shall': 82, 'might': 83, 'time': 84, 'can': 85, 'eyes': 86, 'said': 87, 'how': 88, 'such': 89,

In [ ]:
targets_indexed[0]

[2]

In [ ]:
X[0]

array([   2,  104,   93,  840,    5,  260,   24, 1439, 1440, 1260,   26,
        840,   32,   23,    2,  420,    5, 1672, 3123,   31,   52, 2434,
          3,   13,  207,   52, 3124, 2435,   16,   77,  775,   19,  336,
         19,  157,   35, 1673,  420,   19,  268])

程式6-4


In [ ]:
# 建構並訓練模型
training_model = Sequential()
training_model.add(Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS,
    mask_zero=True, input_length=None))
training_model.add(LSTM(128, return_sequences=True,
                        dropout=0.2, recurrent_dropout=0.2))
training_model.add(LSTM(128, dropout=0.2,
                        recurrent_dropout=0.2))
training_model.add(Dense(128, activation='relu'))
training_model.add(Dense(MAX_WORDS, activation='softmax'))
training_model.compile(loss='categorical_crossentropy',
                       optimizer='adam')
training_model.summary()
history = training_model.fit(X, y, validation_split=0.05,
                             batch_size=BATCH_SIZE,
                             epochs=EPOCHS, verbose=2,
                             shuffle=True)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         750000    
                                                                 
 lstm (LSTM)                 (None, None, 128)         117248    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 7500)              967500    
                                                                 
Total params: 1,982,844
Trainable params: 1,982,844
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
101/101 - 175s - loss: 7.0798 - val_

程式6-5

In [ ]:
# 另建一同架構、但能延續內部狀態的模型，用來預測單字

inference_model = Sequential()
inference_model.add(Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS,
    mask_zero=True, batch_input_shape=(1, 1)))
inference_model.add(LSTM(128, return_sequences=True,
                         dropout=0.2, recurrent_dropout=0.2,
                         stateful=True))
inference_model.add(LSTM(128, dropout=0.2,
                         recurrent_dropout=0.2, stateful=True))
inference_model.add(Dense(128, activation='relu'))
inference_model.add(Dense(MAX_WORDS, activation='softmax'))
weights = training_model.get_weights()
inference_model.set_weights(weights)


程式 6-6

In [ ]:
# 從給定兩單字出發預測後續單字
first_words = ['i', 'saw']
first_words_indexed = tokenizer.texts_to_sequences(
    first_words)
inference_model.reset_states()
predicted_string = ''

# 將初始單字餵入模型
for i, word_index in enumerate(first_words_indexed):
    x = np.zeros((1, 1), dtype=np.int64)
    x[0][0] = word_index[0]
    predicted_string += first_words[i]
    predicted_string += ' '
    y_predict = inference_model.predict(x, verbose=0)[0]

# 預測後續 PREDICT_LENGTH (本例為 3) 個單字
for i in range(PREDICT_LENGTH):
    new_word_index = np.argmax(y_predict)
    word = tokenizer.sequences_to_texts(
        [[new_word_index]])
    x[0][0] = new_word_index
    predicted_string += word[0]
    predicted_string += ' '
    y_predict = inference_model.predict(x, verbose=0)[0]
print(predicted_string)


i saw the the the 


程式6-7

In [ ]:
# 探索詞向量之間的相似程度
embeddings = training_model.layers[0].get_weights()[0]
lookup_words = ['the', 'saw', 'see', 'of', 'and',
                'monster', 'frankenstein', 'read', 'eat']
for lookup_word in lookup_words:
    lookup_word_indexed = tokenizer.texts_to_sequences(
        [lookup_word])
    print('words close to:', lookup_word)
    lookup_embedding = embeddings[lookup_word_indexed[0]]
    word_indices = {}

    # 計算距離
    for i, embedding in enumerate(embeddings):
        distance = np.linalg.norm(
            embedding - lookup_embedding)
        word_indices[distance] = i

   # 依間距由小到大依序顯示
    for distance in sorted(word_indices.keys())[:5]:
        word_index = word_indices[distance]
        word = tokenizer.sequences_to_texts([[word_index]])[0]
        print(word + ': ', distance)
    print('')


words close to: the
the:  0.0
a:  0.40431404
my:  0.43061376
his:  0.44076908
its:  0.5307728

words close to: saw
saw:  0.0
possessed:  0.34513295
among:  0.3488872
him:  0.36001918
replied:  0.3622534

words close to: see
see:  0.0
began:  0.35889566
neck:  0.36474687
exertion:  0.36483356
sides:  0.36614263

words close to: of
of:  0.0
total:  0.34739867
malicious:  0.35271916
fixed:  0.3607156
resumed:  0.36163113

words close to: and
and:  0.0
be:  0.3536584
this:  0.36204997
was:  0.37144232
very:  0.38655546

words close to: monster
monster:  0.0
outcast:  0.34352136
show:  0.3438965
mr:  0.3445968
iliad:  0.3472369

words close to: frankenstein
frankenstein:  0.0
murderous:  0.35492852
imagined:  0.35630438
immeasurably:  0.35779786
pointed:  0.35883242

words close to: read
read:  0.0
side:  0.33238882
lamented:  0.33875918
south:  0.35572055
pursuits:  0.35652775

words close to: eat
eat:  0.0
routine:  0.33272073
elder:  0.33520895
confidence:  0.3363432
owed:  0.33931798

